### Production Features Pipeline

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to the Feature Store at Hopsworks.ai

2. It scrapes the upcoming games for today, and saves the blank records back into the Feature Store at Hopsworks.ai so that they can be accessed by the model for the prediction service via the streamlit app.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. 

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

from src.constants import (
    FEATURE_GROUP_VERSION,
)

DATAPATH = Path(r'data')

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')

    # determine what season we are in currently
    today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    if today.month >= 10:
        SEASON = today.year
    else:
        SEASON = today.year - 1
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 10
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&DateFrom=10/22/23&DateTo=10/29/23


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Sun
[['1610612743', '1610612760'], ['1610612744', '1610612745'], ['1610612737', '1610612749'], ['1610612757', '1610612755'], ['1610612759', '1610612746'], ['1610612747', '1610612758']]
['22300095', '22300096', '22300097', '22300098', '22300099', '22300100']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Access Feature Store**

In [9]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        fs = project.get_feature_store()
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350
Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [10]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg = fs.get_feature_group(
        name="rolling_stats",
        version=FEATURE_GROUP_VERSION,
        )
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [11]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    for j in range(tries):
        try:
            df_old = ds_query.read()
        except KeyError as e:
            if j < tries - 1: 
                time.sleep(10)
                continue
            else:
                raise ValueError('HOPSWORKS failed to connect')
        break

    if df_old.empty:
        if i < tries - 1: 
            time.sleep(10)
        else:
            raise ValueError('HOPSWORKS failed to return data')
    else:
        break



df_old


Finished: Reading data from Hopsworks, using ArrowFlight (1.79s) 


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2023-04-02 00:00:00+00:00,22201165,1610612763,1610612741,2022,107,46.406250,83.312500,44.187500,29,47,128,53.187500,86.375000,29.000000,29,36,0
1,2016-05-01 00:00:00+00:00,41500127,1610612748,1610612766,2015,106,0.482910,0.687988,0.375000,24,58,73,0.321045,1.000000,0.318115,14,36,1
2,2017-12-08 00:00:00+00:00,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
3,2018-05-06 00:00:00+00:00,41700234,1610612740,1610612744,2017,92,0.364014,0.799805,0.154053,17,51,118,0.483887,0.833008,0.333008,28,45,0
4,2013-03-01 00:00:00+00:00,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25708,2007-01-05 00:00:00+00:00,20600483,1610612763,1610612738,2006,119,0.562988,0.792969,0.300049,32,33,128,0.575195,0.691895,0.600098,25,32,0
25709,2009-12-20 00:00:00+00:00,20900395,1610612738,1610612750,2009,122,0.519043,0.829102,0.500000,34,45,104,0.422119,0.828125,0.333008,18,41,1
25710,2010-04-10 00:00:00+00:00,20901187,1610612766,1610612765,2009,99,0.538086,0.846191,0.467041,27,35,95,0.456055,0.653809,0.285889,26,34,1
25711,2010-12-10 00:00:00+00:00,21000334,1610612750,1610612765,2010,109,0.519043,0.817871,0.437988,21,42,99,0.385986,0.860840,0.333008,20,38,1


**Convert Feature Names back to original mixed case**

In [12]:
#hopsworks converts all feature names to lowercase, and for code reuse, we need to convert them back
df_old = convert_feature_names(df_old)
df_old
df_old[df_old['PTS_home'] == 0]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS


**Update Yesterday's Matchup Predictions with New Final Results**

In [13]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2023-04-02 00:00:00+00:00,22201165,1610612763,1610612741,2022,107,46.406250,83.312500,44.187500,29,47,128,53.187500,86.375000,29.000000,29,36,0
1,2016-05-01 00:00:00+00:00,41500127,1610612748,1610612766,2015,106,0.482910,0.687988,0.375000,24,58,73,0.321045,1.000000,0.318115,14,36,1
2,2017-12-08 00:00:00+00:00,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
3,2018-05-06 00:00:00+00:00,41700234,1610612740,1610612744,2017,92,0.364014,0.799805,0.154053,17,51,118,0.483887,0.833008,0.333008,28,45,0
4,2013-03-01 00:00:00+00:00,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25733,2023-10-25 00:00:00,22300064,1610612754,1610612764,2023,143,52.300000,91.700000,46.500000,38,51,120,45.800000,79.300000,37.500000,28,45,1
25734,2023-10-25 00:00:00,22300063,1610612766,1610612737,2023,116,50.000000,73.100000,29.700000,34,51,110,41.900000,81.800000,17.200000,24,42,1
25735,2023-10-25 00:00:00,22300065,1610612752,1610612738,2023,104,37.100000,53.800000,43.900000,24,47,108,48.100000,84.600000,30.800000,18,46,0
25736,2023-10-25 00:00:00,22300068,1610612748,1610612765,2023,103,40.200000,80.800000,36.400000,22,48,102,44.600000,60.000000,34.400000,28,56,1


**Add Today's Matchups for Feature Engineering**

In [14]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [15]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2023-04-02 00:00:00+00:00,22201165,1610612763,1610612741,2022,107.0,46.406250,83.312500,44.187500,29.0,47.0,128.0,53.187500,86.375000,29.000000,29.0,36.0,0.0,0,0.0
1,2016-05-01 00:00:00+00:00,41500127,1610612748,1610612766,2015,106.0,0.482910,0.687988,0.375000,24.0,58.0,73.0,0.321045,1.000000,0.318115,14.0,36.0,1.0,1,1.0
2,2017-12-08 00:00:00+00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0,0,1.0
3,2018-05-06 00:00:00+00:00,41700234,1610612740,1610612744,2017,92.0,0.364014,0.799805,0.154053,17.0,51.0,118.0,0.483887,0.833008,0.333008,28.0,45.0,0.0,1,0.0
4,2013-03-01 00:00:00+00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25739,2023-10-29,22300096,1610612745,1610612744,2023,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
25740,2023-10-29,22300097,1610612749,1610612737,2023,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
25741,2023-10-29,22300098,1610612755,1610612757,2023,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
25742,2023-10-29,22300099,1610612746,1610612759,2023,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0


**Feature Engineering**

In [16]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)



#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28 00:00:00+00:00,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28 00:00:00+00:00,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28 00:00:00+00:00,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29 00:00:00+00:00,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29 00:00:00+00:00,20300008,1610612765,1610612754,2003,87,0.392090,0.742188,0.333008,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25721,2023-10-29 00:00:00+00:00,22300098,1610612755,1610612757,2023,0,0.000000,0.000000,0.000000,0,...,2.782812,6.088542,1.666667,-5.000000,-4.8,-1.933333,0.333333,1.857143,2.9,5.533333
25722,2023-10-29 00:00:00+00:00,22300096,1610612745,1610612744,2023,0,0.000000,0.000000,0.000000,0,...,-3.368750,-4.891667,-4.333333,-7.285714,-6.4,-8.400000,-5.000000,-1.142857,-2.0,-3.400000
25723,2023-10-29 00:00:00+00:00,22300097,1610612749,1610612737,2023,0,0.000000,0.000000,0.000000,0,...,7.618750,4.083333,-4.000000,1.142857,1.2,0.800000,4.666667,-0.571429,-0.8,-1.666667
25724,2023-10-29 00:00:00+00:00,22300099,1610612746,1610612759,2023,0,0.000000,0.000000,0.000000,0,...,3.162500,5.230208,-6.666667,-7.714286,-5.9,-3.333333,-10.000000,-8.285714,-3.9,-4.733333


**Insert New Data into Feature Group**

In [17]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg.insert(df_combined, overwrite = True, write_options={"wait_for_job" : False})
    except KeyError as e:
        if i < tries - 1: 
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Uploading Dataframe: 0.00% |          | Rows 0/25726 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: rolling_stats_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_2_offline_fg_materialization/executions


In [18]:
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28 00:00:00+00:00,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28 00:00:00+00:00,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28 00:00:00+00:00,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29 00:00:00+00:00,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29 00:00:00+00:00,20300008,1610612765,1610612754,2003,87,0.392090,0.742188,0.333008,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25721,2023-10-29 00:00:00+00:00,22300098,1610612755,1610612757,2023,0,0.000000,0.000000,0.000000,0,...,2.782812,6.088542,1.666667,-5.000000,-4.8,-1.933333,0.333333,1.857143,2.9,5.533333
25722,2023-10-29 00:00:00+00:00,22300096,1610612745,1610612744,2023,0,0.000000,0.000000,0.000000,0,...,-3.368750,-4.891667,-4.333333,-7.285714,-6.4,-8.400000,-5.000000,-1.142857,-2.0,-3.400000
25723,2023-10-29 00:00:00+00:00,22300097,1610612749,1610612737,2023,0,0.000000,0.000000,0.000000,0,...,7.618750,4.083333,-4.000000,1.142857,1.2,0.800000,4.666667,-0.571429,-0.8,-1.666667
25724,2023-10-29 00:00:00+00:00,22300099,1610612746,1610612759,2023,0,0.000000,0.000000,0.000000,0,...,3.162500,5.230208,-6.666667,-7.714286,-5.9,-3.333333,-10.000000,-8.285714,-3.9,-4.733333


In [19]:
# check to make sure there are no duplicate games were inadvertently added
df_combined[df_combined.duplicated(subset=['GAME_ID'], keep=False)]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
